In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd

from models.functions.data_loader import GeologyTracesDataset
from models.functions.fno_model import FNO_3D
from models.functions.uno_model import UNO_3D
from models.functions.gfno_model import GFNO_3D
from models.functions.ffno_model import FFNO_3D
from models.functions.intensity_measures import *

This notebook load each trained model and plot the predictions. 

In [2]:
S_in = 32 # Number of spatial points in inputs (dimensions x and y)
S_in_z = 32 # Number of vertical points in inputs (dimension z)
S_out = 32 # Number of spatial points in outputs (dimensions x and y)
T_out = 320 # Number of timesteps
dt = 0.02 # Time step in seconds

Load data

In [3]:
Nval = 5 # Number of validation samples
val_data = GeologyTracesDataset('./models/inputs/', ['inputs3D_S32_Z32_T320_fmax5_val'], 
                                S_in=S_in, S_in_z=S_in_z, S_out=S_out, T_out=T_out,
                                transform_a='normal', N=Nval)

val_loader = torch.utils.data.DataLoader(val_data,
                                         batch_size=1,
                                         shuffle=False)

# Load models

You should load only one model by running its corresponding cell, and then plot its predictions. 

### FNO model

In [4]:
dv = 16 # Number of channels after the uplift
in_width = 6 # Number of channels after the grid concatenation
list_dv = [16, 16, 16, 16] # Number of channels after each block
list_D1 = [32, 32, 32, 32] # Dimensions along the 1st dimension after each block
list_D2 = [32, 32, 32, 32] # Dimensions along the 2nd dimension after each block
list_D3 = [64, 128, 256, 320] # Dimensions along the 3rd dimension after each block
list_M1 = [16, 16, 16, 16] # Number of modes along the 1st dimension after each block
list_M2 = [16, 16, 16, 16] # Number of modes along the 2nd dimension after each block
list_M3 = [16, 32, 32, 32] # Number of modes along the 3rd dimension after each block
model_name = 'FNO'

# build the model
model = FNO_3D(in_width, dv, S_in, list_dv, list_D1, list_D2, list_D3, list_M1, list_M2, list_M3, padding=0)

# load the weights from the trained model
epochs = 158
name_config = f'FNO3D-dv{dv}-S{S_in}-T{T_out}-learningrate0p0006-L1loss1p0-L2loss0p0-Ntrain27000-batchsize16'
model.load_state_dict(torch.load(f'./models/logs/models/bestmodel-{name_config}-epochs{epochs}.pt', map_location='cpu'))
model.eval()
print()

### UNO model

In [5]:
dv = 16 # Number of channels after the uplift
in_width = 6 # Number of channels after the grid concatenation
list_dv = [16, 16, 16, 16, 16, 16, 16, 16] # Number of channels after each block
list_D1 = [24, 18, 13, 8, 13, 18, 24, 32] # Dimensions along the 1st dimension after each block
list_D2 = [24, 18, 13, 8, 13, 18, 24, 32] # Dimensions along the 2nd dimension after each block
list_D3 = [24, 18, 13, 8, 17, 34, 64, 320] # Dimensions along the 3rd dimension after each block
list_M1 = [12, 9, 6, 4, 4, 6, 9, 12, 12] # Number of modes along the 1st dimension after each block
list_M2 = [12, 9, 6, 4, 4, 6, 9, 12, 12] # Number of modes along the 2nd dimension after each block
list_M3 = [12, 9, 7, 5, 5, 9, 17, 20, 20] # Number of modes along the 3rd dimension after each block
model_name = 'U-NO'

# build the model
model = UNO_3D(in_width, dv, S_in, list_dv, list_D1, list_D2, list_D3, list_M1, list_M2, list_M3, padding=0)

# load the weights from the trained model
epochs = 200
name_config = f'UNO3D-dv{dv}-S{S_in}-T{T_out}-learningrate0p0006-L1loss1p0-L2loss0p0-Ntrain27000-batchsize16'
model.load_state_dict(torch.load(f'./models/logs/models/bestmodel-{name_config}-epochs{epochs}.pt', map_location='cpu'))
model.eval()
print()

### G-FNO model

In [6]:
dv = 11 # Number of channels after the uplift
num_channels = 1 # Number of channels in inputs
list_dv = [16, 16, 16, 16] # Number of channels after each block
list_D = [32, 32, 32, 32] # Dimensions along the 1st and 2nd dimension after each block
list_D3 = [64, 128, 256, 320] # Dimensions along the 3rd dimension after each block
list_M = [8, 8, 8, 8] # Number of modes along the 1st and 2nd dimension after each block
list_M3 = [8, 8, 8, 8] # Number of modes along the 3rd dimension after each block
model_name = 'G-FNO'

# build the model
model = model = GFNO_3D(num_channels, T_out, list_D, list_D3, list_M, list_M3, width=dv, padding=0, 
                        initial_step=1, reflection=False, grid_type='cartesian')

# load the weights from the trained model
epochs = 182
name_config = f'GFNO3D-dv{dv}-S{S_in}-T{T_out}-padding0-learningrate0p0001-L1loss1p0-L2loss0p0-Ntrain27000-batchsize16'\
'-modes8-modestime8'
model.load_state_dict(torch.load(f'./models/logs/models/bestmodel-{name_config}-epochs{epochs}.pt', map_location='cpu'))
model.eval()
print()

### F-FNO model

In [4]:
nlayers = 28 # Number of layers
dv = 16 # Number of channels after the uplift
list_dv = [16]*nlayers # Number of channels after each block
list_D1 = [32]*nlayers # Dimensions along the 1st dimension after each block
list_D2 = [32]*nlayers # Dimensions along the 2nd dimension after each block
list_D3 = [32]*(nlayers-4) + [64, 128, 256, 320] # Dimensions along the 3rd dimension after each block
list_M1 = [16]*nlayers # Number of modes along the first dimension after each block
list_M2 = [16]*nlayers # Number of modes along the first dimension after each block
list_M3 = [16]*(nlayers-4) + [16, 32, 32, 32] # Number of modes along the first dimension after each block
model_name = 'F-FNO'

# build the model
model = FFNO_3D(list_D1, list_D2, list_D3,
                list_M1, list_M2, list_M3, dv, 
                input_dim=4, # to define the uplift network (last dimension after grid concatenation)
                output_dim=1, # to define the projection network (last dimension after projection)
                n_layers=nlayers, padding = 0)

# load the weights from the trained model
epochs = 350
name_config = f'FFNO3D-dv{dv}-{nlayers}layers-S{S_in}-T{T_out}-padding0-learningrate0p0006-L1loss1p0-L2loss0p0'\
'-Ntrain27000-batchsize16'
model.load_state_dict(torch.load(f'./models/logs/models/bestmodel-{name_config}-epochs{epochs}.pt', map_location='cpu'))
model.eval()
print()

# Intensity Measures

Compute the outputs for all samples

In [5]:
device = 'cpu'
all_uE = np.zeros((Nval, S_out, S_out, T_out), dtype=np.float32)
all_outE = np.zeros((Nval, S_out, S_out, T_out), dtype=np.float32)

all_uZ = np.zeros((Nval, S_out, S_out, T_out), dtype=np.float32)
all_outZ = np.zeros((Nval, S_out, S_out, T_out), dtype=np.float32)

all_uN = np.zeros((Nval, S_out, S_out, T_out), dtype=np.float32)
all_outN = np.zeros((Nval, S_out, S_out, T_out), dtype=np.float32)

i = 0
with torch.no_grad():        
    for _ in val_loader:
        a = _[0].to(device)
        uE = _[1].to(device)
        uZ = _[2].to(device)
        uN = _[3].to(device)

        outE, outZ, outN = model(a)
        all_uE[i:i+a.shape[0]] = uE.cpu().numpy()[:, :, :, :, 0]
        all_outE[i:i+a.shape[0]] = outE.cpu().numpy()[:, :, :, :, 0]
        all_uZ[i:i+a.shape[0]] = uZ.cpu().numpy()[:, :, :, :, 0]
        all_outZ[i:i+a.shape[0]] = outZ.cpu().numpy()[:, :, :, :, 0]
        all_uN[i:i+a.shape[0]] = uN.cpu().numpy()[:, :, :, :, 0]
        all_outN[i:i+a.shape[0]] = outN.cpu().numpy()[:, :, :, :, 0]
        i += a.shape[0]

Compute intensity measures

In [6]:
Nx = all_uE.shape[1] # number of sensors in the horizontal direction
Ny = all_uE.shape[2] # number of sensors in the vertical direction
Nt = all_uE.shape[3] # number of time steps

all_rRMSE = dict()
all_rRSD = dict()
all_rCAV = dict()
all_rPGV = dict()
all_rFFTlow = dict()
all_rFFTmid = dict()
all_rFFThigh = dict()

for c in ['E', 'N', 'Z']: # iterate over all components
    if c=='E':
        u = all_uE.copy()
        out = all_outE.copy()

    elif c=='N':
        u = all_uN.copy()
        out = all_outN.copy()

    elif c=='Z':
        u = all_uZ.copy()
        out = all_outZ.copy()
        
    ### MEAN ERROR
    eps = 1e-4
    all_rRMSE[c] = np.sqrt(np.mean((u - out)**2/(u**2 + eps), axis=(1,2,3)))

    u = u.reshape(Nval*Nx*Ny, Nt)
    out = out.reshape(Nval*Nx*Ny, Nt)

    ### RELATIVE SIGNIFICANT DURATION
    Aint_u = Arias_integral(u, dt=dt)
    Aint_out = Arias_integral(out, dt=dt)

    RSD_u = relative_significant_duration(Aint_u, dt=dt)
    RSD_out = relative_significant_duration(Aint_out, dt=dt)

    all_rRSD[c] = score_metric(RSD_u, RSD_out, method='bias') # one score per sample and per sensor

    ### CUMULATIVE ABSOLUTE VELOCITY
    CAV_u = cumulative_absolute_velocity(u, dt=dt)
    CAV_out = cumulative_absolute_velocity(out, dt=dt)
    all_rCAV[c] = score_metric(CAV_u, CAV_out, method='bias')

    ### PEAK GROUND VELOCITY
    PGV_u = np.max(np.abs(u), axis=1)
    PGV_out = np.max(np.abs(out), axis=1)
    all_rPGV[c] = score_metric(PGV_u, PGV_out, method='bias')

    ### FOURIER SPECTRA
    low_freq_u, mid_freq_u, high_freq_u = fourier_spectra(u, (0.1, 0.5), (0.75, 1.5), (2.5, 3.5), dt=dt)
    low_freq_out, mid_freq_out, high_freq_out = fourier_spectra(out, (0.1, 0.5), (0.75, 1.5), (2.5, 3.5), dt=dt)

    all_rFFTlow[c] = score_metric(low_freq_u, low_freq_out, method='bias')
    all_rFFTmid[c] = score_metric(mid_freq_u, mid_freq_out, method='bias')
    all_rFFThigh[c] = score_metric(high_freq_u, high_freq_out, method='bias')

Compute the mean of the three components and print the mean and standard deviation of all samples and all spatial points (note that we are considering only 5 validation samples here, explaining why the results are different from the paper). 

In [7]:
for key, metric in zip(['rRMSE', 'rRSD', 'rCAV', 'rPGV', 'rFFTlow', 'rFFTmid', 'rFFThigh'],
                       [all_rRMSE, all_rRSD, all_rCAV, all_rPGV, all_rFFTlow, all_rFFTmid, all_rFFThigh]):
    mean_metric = (metric['E'] + metric['N'] + metric['Z'])/3
    print(f"{key}: mean={np.mean(mean_metric):.3f} - std={np.std(mean_metric):.3f}")

rRMSE: mean=0.169 - std=0.058
rRSD: mean=-0.169 - std=0.357
rCAV: mean=-0.425 - std=0.159
rPGV: mean=-0.136 - std=0.168
rFFTlow: mean=-0.006 - std=0.315
rFFTmid: mean=-0.106 - std=0.390
rFFThigh: mean=-0.091 - std=0.638
